# 1. 임베딩

In [2]:
# 1. 희소 표현 기반 임베딩
import pandas as pd
class2 = pd.read_csv('/content/drive/MyDrive/딥러닝 파이토치 실습/10. 자연어 처리를 위한 임베딩/class2.csv', index_col=0)

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

train_x = label_encoder.fit_transform(class2['class2'])
train_x

array([2, 2, 1, 0, 1, 0])

In [3]:
# 2. 횟수 기반 임베딩
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is last chance.',
    'and if you do not have this chance.',
    'you will never get any chance.',
    'will you do get this one?',
    'please, get this chance',
]

vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'this': 13,
 'is': 7,
 'last': 8,
 'chance': 2,
 'and': 0,
 'if': 6,
 'you': 15,
 'do': 3,
 'not': 10,
 'have': 5,
 'will': 14,
 'never': 9,
 'get': 4,
 'any': 1,
 'one': 11,
 'please': 12}

In [4]:
# 배열 변환
vect.transform(['you will never get any chance.']).toarray()

array([[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]])

In [5]:
# 불용어를 제거한 카운터 벡터
vect = CountVectorizer(stop_words=['and', 'is', 'please', 'this']).fit(corpus)
vect.vocabulary_

{'last': 6,
 'chance': 1,
 'if': 5,
 'you': 11,
 'do': 2,
 'not': 8,
 'have': 4,
 'will': 10,
 'never': 7,
 'get': 3,
 'any': 0,
 'one': 9}

In [6]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

doc = ['I like machine learning', 'I love deep learning', 'I run everyday']

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)

doc_distance = (tfidf_matrix * tfidf_matrix.T)

print('유사도를 위한', str(doc_distance.get_shape()[0]), 'x', str(doc_distance.get_shape()[1]), 'matrix를 만들었습니다.')
print(doc_distance.toarray())

유사도를 위한 3 x 3 matrix를 만들었습니다.
[[1.       0.224325 0.      ]
 [0.224325 1.       0.      ]
 [0.       0.       1.      ]]


In [7]:
# 3. 예측 기반 임베딩
!pip install nltk

import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
# 3.1 word2vec
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec

from tqdm import tqdm_notebook

sample = open('/content/drive/MyDrive/딥러닝 파이토치 실습/10. 자연어 처리를 위한 임베딩/peter.txt', 'r', encoding='utf-8')
s = sample.read()

f = s.replace('\n', ' ')

data = []
for i in tqdm_notebook(sent_tokenize(f)): # 로딩한 파일의 각 문장마다 반복
  temp = []
  for j in word_tokenize(i): # 문장을 단어로 토큰화
    temp.append(j.lower()) # 토큰화된 단어를 소문자로 변환하여 temp에 저장

  data.append(temp)

len(data), data[0]

<ipython-input-8-69fc6ed289ae>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(sent_tokenize(f)): # 로딩한 파일의 각 문장마다 반복


  0%|          | 0/89 [00:00<?, ?it/s]

(89,
 ['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'])

In [9]:
# cbow
model1 = gensim.models.Word2Vec(data, min_count=1, size=100, window=5, sg=0)

print("Cosine similarity between 'peter' " + "wendy' - CBOW : ", model1.wv.similarity('wendy', 'wendy'))
print("Cosine similarity between 'peter' " + "hook' - CBOW : ",  model1.wv.similarity('peter', 'hook')) 

Cosine similarity between 'peter' wendy' - CBOW :  1.0000001
Cosine similarity between 'peter' hook' - CBOW :  -0.05421149


In [10]:
# skip-gram
model2 = gensim.models.Word2Vec(data, min_count=1, size=100, window=5, sg=1)

print("Cosine similarity between 'peter' " + "wendy' - CBOW : ", model2.wv.similarity('wendy', 'wendy'))
print("Cosine similarity between 'peter' " + "hook' - CBOW : ",  model2.wv.similarity('peter', 'hook')) 

Cosine similarity between 'peter' wendy' - CBOW :  1.0
Cosine similarity between 'peter' hook' - CBOW :  0.47784954


In [11]:
# 3.2 fasttext
from gensim.test.utils import common_texts
from gensim.models.fasttext import FastText as ft_gensim

import warnings
warnings.filterwarnings('ignore')

stemmed = ['database', 'science', 'scientist', 'mgmt', 'microsoft', 'hire', 'develop', 'mentor', 'team', 'data', 'scientist', 'define', 'dataloader', 'scienc', 'priority', 'deep', 'understand', 
           'learn', 'goal', 'collabor', 'across', 'triple', 'group', 'set', 'team', 'shortterm', 'longterm', 'goal', 'act', 'strait', 'advisor', 'leadership', 'influenc', 'future', 'direct', 
           'strategy', 'define', 'partnership', 'align', 'effect', 'broad', 'analyt', 'effort', 'analyticsdata', 'team', 'drive', 'part', 'datadog', 'scienc', 'bi', 'common', 'disciplin', 
           'microsoftprior', 'experi', 'hire', 'manage', 'runner', 'team', 'data', 'scientist', 'busi', 'domain', 'experi', 'usage', 'analyt', 'must', 'experi', 'across', 'sever', 'relev', 
           'busi', 'domain', 'util', 'critic', 'think', 'skill', 'concept', 'complex', 'busi', 'problem', 'salt', 'use', 'advanc', 'analsis', 'large', 'scale', 'realworld', 'busi', 'data', 
           'set', 'candid', 'must', 'abl', 'independ', 'execut', 'analyt', 'project', 'help', 'intern', 'client', 'understand']
def gen_words(stemmed):
    yield stemmed

model = ft_gensim(size=100, window=5, min_count=1, workers=4, sg=1)
model.build_vocab(gen_words(stemmed))

model.train(gen_words(stemmed), total_examples=model.corpus_count, epochs=model.iter)
model.wv.most_similar(positive=['scientist'])

[('scienc', 0.40221747756004333),
 ('science', 0.35087889432907104),
 ('must', 0.3002471923828125),
 ('problem', 0.23477956652641296),
 ('part', 0.22043737769126892),
 ('experi', 0.1733037531375885),
 ('data', 0.1570461541414261),
 ('bi', 0.12951719760894775),
 ('direct', 0.1187698021531105),
 ('broad', 0.10711653530597687)]

In [12]:
from gensim.models import KeyedVectors

model_kr = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/딥러닝 파이토치 실습/10. 자연어 처리를 위한 임베딩/wiki.ko.vec')

find_similar_to = '노력'
for similar_word in model_kr.similar_by_word(find_similar_to):
  print("Word: {0}, Similarity: {1:.2f}".format(similar_word[0], similar_word[1]))

Word: 노력함, Similarity: 0.80
Word: 노력중, Similarity: 0.75
Word: 노력만, Similarity: 0.72
Word: 노력과, Similarity: 0.71
Word: 노력의, Similarity: 0.69
Word: 노력가, Similarity: 0.69
Word: 노력이나, Similarity: 0.69
Word: 노력없이, Similarity: 0.68
Word: 노력맨, Similarity: 0.68
Word: 노력보다는, Similarity: 0.68


In [13]:
similarities = model_kr.most_similar(positive=['동물', '육식동물'], negative=['사람'])
print(similarities)

[('초식동물', 0.7804122567176819), ('거대동물', 0.7547270059585571), ('육식동물의', 0.7547166347503662), ('유두동물', 0.7535113096237183), ('반추동물', 0.7470757961273193), ('독동물', 0.7466292381286621), ('육상동물', 0.746031641960144), ('유즐동물', 0.7450904846191406), ('극피동물', 0.7449344992637634), ('복모동물', 0.7424346208572388)]


In [14]:
# 4. 횟수 / 예측 기반 임베딩
# Glove
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [24]:
glove_file = datapath('/content/drive/MyDrive/딥러닝 파이토치 실습/10. 자연어 처리를 위한 임베딩/glove.6B.100d.txt')  #구글 드라이브에서 파일 경로 복사 후 붙여넣으세요. 
# glove_file = datapath('glove.6B.100d.txt')                                        # PC에서 불러온다면 주석 해제 후 실습
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 100)

In [25]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)
model.most_similar('bill')

[('legislation', 0.8072140216827393),
 ('proposal', 0.7306863069534302),
 ('senate', 0.7142540812492371),
 ('bills', 0.7044401168823242),
 ('measure', 0.6958035230636597),
 ('passed', 0.6906244158744812),
 ('amendment', 0.6846879720687866),
 ('provision', 0.6845567226409912),
 ('plan', 0.6816462874412537),
 ('clinton', 0.6663139462471008)]

In [26]:
model.most_similar('cherry')

[('peach', 0.688809871673584),
 ('mango', 0.6838189959526062),
 ('plum', 0.6684104204177856),
 ('berry', 0.6590359210968018),
 ('grove', 0.6581551432609558),
 ('blossom', 0.6503506302833557),
 ('raspberry', 0.6477391719818115),
 ('strawberry', 0.6442098617553711),
 ('pine', 0.6390928626060486),
 ('almond', 0.6379213333129883)]

In [27]:
model.most_similar(negative=['cherry'])

[('kazushige', 0.4834350347518921),
 ('askerov', 0.4778186082839966),
 ('lakpa', 0.46915262937545776),
 ('ex-gay', 0.45713329315185547),
 ('tadayoshi', 0.4522106647491455),
 ('turani', 0.4481006860733032),
 ('saglam', 0.446959912776947),
 ('aijun', 0.4435269832611084),
 ('adjustors', 0.44235295057296753),
 ('nyum', 0.4423118233680725)]

In [28]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [30]:
def analogy(x1, x2, y1):
    result = model.most_similar(positive=[y1, x2], negative=[x1])
    return result[0][0]
    
analogy('australia', 'beer', 'france')

'champagne'

In [31]:
analogy('tall', 'tallest', 'long')

'longest'

In [32]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal
